In [ ]:
import pandas as pd
import string
import nltk
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, precision_recall_curve, accuracy_score
from sklearn.metrics import roc_auc_score
!pip install imblearn --quiet

import warnings

warnings.filterwarnings('ignore')
nltk.download('punkt')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Agent_LLM_paper')

In [ ]:
from models import InferSent

In [ ]:
nltk.download('stopwords')

In [ ]:
!pip install torch

In [ ]:
import torch

In [ ]:
MODEL_PATH ="/content/drive/MyDrive/Agent_LLM_paper/infersent1.pkl"
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': 1}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [ ]:
use_cuda = False
infersent = infersent.cuda() if use_cuda else infersent

In [ ]:
W2V_PATH = '/content/drive/MyDrive/Agent_LLM_paper/glove.6B.300d.txt'
infersent.set_w2v_path(W2V_PATH)

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/Agent_LLM_paper/drugsComTest_raw.tsv',sep = '\t')
train_data = pd.read_csv('/content/drive/MyDrive/Agent_LLM_paper/drugsComTrain_raw.tsv',sep = '\t')

In [ ]:
test_data['label'] = test_data['rating'].apply(lambda x: 0 if x <= 5 else 1)
train_data['label'] = train_data['rating'].apply(lambda x: 0 if x <= 5 else 1)

In [ ]:
train_data.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,label
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,1
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,1
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,0
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,1
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,1


In [ ]:
test_data['clean_review'] = test_data['review'].str.lower()
train_data['clean_review'] = train_data['review'].str.lower()

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in STOPWORDS])
def remove_spl_chars(text):
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    text = re.sub('\s+', ' ', text)
    return text

train_data['clean_review'] = train_data['clean_review'].apply(lambda x: remove_stopwords(x))
train_data['clean_review'] = train_data['clean_review'].apply(lambda x: remove_spl_chars(x))

test_data['clean_review'] = test_data['clean_review'].apply(lambda x: remove_stopwords(x))
test_data['clean_review'] = test_data['clean_review'].apply(lambda x: remove_spl_chars(x))

In [ ]:
df_total = pd.concat([train_data, test_data], axis=0)

In [ ]:
infersent.build_vocab(df_total.clean_review, tokenize=True)

Found 32219(/53570) words with w2v vectors
Vocab size : 32219


In [ ]:
x_train = infersent.encode(train_data.clean_review.tolist(), bsize=128, tokenize=True)

161297


In [ ]:
x_test= infersent.encode(test_data.clean_review.tolist(), bsize=128, tokenize=True)

53766


In [ ]:
import pickle

In [ ]:
with open('/content/drive/MyDrive/Agent_LLM_paper/x_train_infersent.pkl', 'wb') as fichier:
    pickle.dump(x_train, fichier)

with open('/content/drive/MyDrive/Agent_LLM_paper/x_test_infersent.pkl', 'wb') as fichier:
    pickle.dump(x_test, fichier)